In [ ]:
import math
reported_data = {}
def populate_reported_data():
    all_values = [74.76, 69.06, 69.07, 81.38, 72.66, 72.70, 78.77, 70.52, 70.55, 84.58, 75.05, 75.05, 84.71, 76.36, 76.38, 67.62, 63.77, 62.51, 80.21, 79.06, 77.01, 81.48, 79.42, 77.83, 77.71, 77.87, 75.71, 89.51, 85.55, 84.69, 89.85, 85.05, 84.14, 47.20, 70.10, 57.78, 74.43, 72.64, 70.93, 76.18, 73.52, 73.81, 74.65, 72.75, 69.37, 79.95, 74.97, 75.16, 81.58, 76.08, 73.85, 71.28, 71.72, 63.40, 72.27, 63.31, 63.55, 73.72, 65.49, 65.67, 73.18, 65.17, 64.64, 78.04, 68.97, 69.10, 79.34, 69.87, 69.93, 69.22, 64.60, 63.50, 69.83, 63.11, 62.49, 70.07, 63.35, 62.49, 70.11, 63.49, 62.85, 71.00, 63.00, 60.78, 72.09, 64.35, 63.73, 65.65, 59.99, 58.80, 63.15, 83.88, 77.26, 66.20, 83.90, 77.32, 66.09, 83.97, 77.56, 64.2, 84.01, 77.67, 66.65, 84.00, 77.65, 64.62, 82.90, 75.60, 70.73, 71.66, 66.24, 76.90, 75.76, 72.46, 76.81, 74.92, 72.02, 71.35, 71.87, 65.53, 77.42, 75.55, 71.78, 72.44, 74.00, 70.02, 61.69, 55.64, 53.48, 62.49, 56.79, 54.37, 61.87, 54.85, 51.54, 78.80, 72.51, 72.20, 79.12, 72.75, 72.54, 64.27, 53.53, 37.33, 81.98, 80.94, 80.10, 81.95, 80.95, 80.10, 81.97, 80.95, 80.10, 82.17, 81.28, 80.52, 82.13, 81.29, 80.54, 68.18, 64.49, 58.59]
    datasets = ["BPIC2011-f1","BPIC2011-f2","BPIC2011-f3","BPIC2011-f4","BPIC2012-ac","BPIC2012-re","BPIC2012-ca","Production","Traffic Fine"]
    encodings = ["OH_ACT","WOH_BIN","WOH_FREQ","EMB_ACT","EMB_ACT+LPM","EMB_LPM"]
    metrics = ["roc-auc","accuracy","w_f1"]
    print(len(all_values))
    for i in range(len(all_values)):
        dataset_index = max([0,math.floor(i/(len(metrics)*len(encodings)))])
        d = datasets[dataset_index]
        e = encodings[math.floor(i/len(metrics))%len(encodings)]
        m = metrics[i%len(metrics)]
        print(d,e,m,": ", all_values[i])
        reported_data[(d,e,m)] = all_values[i]/100.0
populate_reported_data()

In [ ]:
from collections import defaultdict
import json
import glob
import subprocess
from matplotlib import pyplot as plt
import seaborn as sns
main_data_folder = "../data/main"
alternative_data_folder = "../data/imf-alternative"
figure_path = "../figures/"
plt.rc('font', size=16)
logs = {
    "BPIC11_f1": "BPIC2011-f1",
    "BPIC11_f2": "BPIC2011-f2",
    "BPIC11_f3": "BPIC2011-f3",
    "BPIC11_f4": "BPIC2011-f4",
    "Production": "Production",
    "traffic_fines_1": "Traffic Fine",
        }
all_results = {}
alt_all_results = {}
for log_name in logs:
    filename = f"{main_data_folder}/results/{log_name}/"
    alt_filename = f"{main_data_folder}/results/{log_name}/"
    all_results[log_name] = dict()
    alt_all_results[log_name] = dict()
    for file in glob.glob(filename + "*_info.json"):
        mode = "_".join(file.split("/")[-1].split("_")[-3:-1])
        with open(file, "r") as f:
            json_data = json.load(f)
            mode_data = all_results[log_name].get(mode,[])
            mode_data.append(json_data)
            all_results[log_name][mode] = mode_data
    for file in glob.glob(alt_filename + "*_info.json"):
        mode = "_".join(file.split("/")[-1].split("_")[-3:-1])
        with open(file, "r") as f:
            json_data = json.load(f)
            mode_data = alt_all_results[log_name].get(mode,[])
            mode_data.append(json_data)
            alt_all_results[log_name][mode] = mode_data
    print(all_results[log_name].keys())
    measure_labels = {"roc-auc": "AUC", "accuracy": "Accuracy", "w_f1": "F1-score"} 
encodings = {"OH_ACT": "$OH_{Act}$","WOH_BIN": "$WOH_{Bin}$","WOH_FREQ": "$WOH_{Freq}$","EMB_ACT": "$Emb_{Act}$","EMB_ACT+LPM": "$Emb_{Act+LPMs}$","EMB_LPM": "$Emb_{LPMs}$"}


In [ ]:

import csv
import datetime

import numpy as np


time_stamp = datetime.datetime.now().isoformat()

def format_and_round(value):
    return "{:2.2f}".format(value) + "\%"

conf_fact = 3

with open(
        f"../table-data_{time_stamp}.csv",
        "w",
        newline="",
    ) as f:
        w = csv.writer(f)
        for log_name in logs:
            long_log_name = logs[log_name]
            for enc in encodings:
                row = []
                for measure in measure_labels:
                    values = [v['test_res'][measure] for v in all_results[log_name][enc]]
                    values = [v for v in values if v != 0.0]
                    reported_value = reported_data[(long_log_name,enc,measure)]
                    sd = np.std(values)
                    mean = np.mean(values)
                    interesting = 'no'
                    print(mean,mean-conf_fact * sd,reported_value)
                    if reported_value >= mean - conf_fact * sd and reported_value <= mean + conf_fact * sd:
                         pass
                    elif reported_value >= mean - conf_fact * sd:
                        interesting = 'above'
                    else:
                        interesting = 'below'
                    maxi = np.max(values)
                    median = np.median(values)
                    formatted_reported = format_and_round(reported_value*100.0)
                    if interesting == 'above':
                         formatted_reported = f"\\textbf{{{formatted_reported}}}"
                    row.append(formatted_reported)
                    row.append(format_and_round(maxi*100.0))
                    formatted_mean = format_and_round(mean*100.0)
                    if interesting == 'below':
                         print("!")
                         formatted_mean = f"\\textbf{{{formatted_mean}}}"
                    row.append(formatted_mean)
                    row.append(format_and_round(median*100.0))
                    row.append(format_and_round(sd*100.0))
                w.writerow(row)


In [ ]:
import numpy as np
in_confidence_interval = 0
higher_than_conf_int = 0
lower_than_conf_int = 0
confidence_interval_factor = 3
mode_confidence_data = {}
for mode in encodings:
    mode_confidence_data[mode] = {
        "in": 0,
        "higher": 0,
        "lower": 0
    }
alt_mode_confidence_data = {}
for mode in encodings:
    alt_mode_confidence_data[mode] = {
        "in": 0,
        "higher": 0,
        "lower": 0
    }
confidence_interval_percentage = "$>68\%$" if confidence_interval_factor == 1 else ("$>95\%$" if confidence_interval_factor == 2 else ("$>99\%$" if confidence_interval_factor >= 3 else "??"))
for log_name in logs:
    full_log_name = logs[log_name]
    for mode in all_results[log_name]:
        for measure in measure_labels:
            print(measure + " for " + mode)
            raw_values = [v['test_res'][measure] for v in all_results[log_name][mode]]
            filtered_values = [v for v in raw_values if v != 0]
            mean = np.mean(filtered_values)
            std = np.std(filtered_values)

            # raw_values_alt = [v['test_res'][measure] for v in alt_all_results[log_name][mode]]
            # for v in raw_values_alt:
            #     if v >= mean - confidence_interval_factor * std and v <= mean + confidence_interval_factor * std:
            #         alt_mode_confidence_data[mode]["in"] += 1
            #     elif v >= mean - confidence_interval_factor * std:
            #         alt_mode_confidence_data[mode]["higher"] += 1
            #     else:
            #         alt_mode_confidence_data[mode]["lower"] += 1
            # filtered_values_alt = [v for v in raw_values if v != 0]
            reported_value = reported_data[(full_log_name,mode,measure)]
            if reported_value >= mean - confidence_interval_factor * std and reported_value <= mean + confidence_interval_factor * std:
                in_confidence_interval += 1
                mode_confidence_data[mode]["in"] += 1
            elif reported_value >= mean - confidence_interval_factor * std:
                higher_than_conf_int += 1
                mode_confidence_data[mode]["higher"] += 1
            else:
                lower_than_conf_int += 1
                mode_confidence_data[mode]["lower"] += 1
            sns.boxplot(filtered_values, orient="h",color="#ccdafc50", whis=99999999)
            plt.scatter(raw_values,[0 for acc in raw_values],label="measured values", alpha=0.8)
            plt.scatter([mean],[0],label="measured mean ($\mu$)", alpha=0.8, marker="+", sizes=[100], color="green")
            # plt.scatter(raw_values_alt,[0 for acc in raw_values],label="measured values\n(alternative LPM discovery)", alpha=0.8, color="red")
            plt.scatter([reported_value],[0],label="reported value", alpha=0.8)
            ax = plt.gca()
            f = plt.gcf()
            f.set_figwidth(15)
            f.set_figheight(2.5)
            r_width = std * 2 * confidence_interval_factor
            r_height = 0.25
            ax.add_patch(plt.Rectangle((mean - r_width/2,-r_height/2),r_width,r_height, alpha=0.3, label=f"$\mu \pm {confidence_interval_factor}\sigma$ ({confidence_interval_percentage} for ND)", color="pink"))
            ax.legend(loc=(1.01, 0.2))
            ax.set_title(f"{measure_labels[measure]} for {full_log_name} with encoding {encodings[mode]}")
            ax.set_xlim([0.0,1.0])
            ax.get_yaxis().set_visible(False)
            svg_path = f"{figure_path}/{measure} for {full_log_name} with encoding {mode}_error.pdf"
            plt.savefig(svg_path, bbox_inches='tight')
            plt.show()

In [ ]:
print(in_confidence_interval,lower_than_conf_int,higher_than_conf_int)
print(in_confidence_interval/(in_confidence_interval + lower_than_conf_int + higher_than_conf_int))
for m in encodings:
    print(m,mode_confidence_data[m])

In [ ]:
for m in encodings:
    print(m,alt_mode_confidence_data[m])
better_interval_dir_names = {"in": "similar","higher":"better", "lower":"worse"}
alt_confidence_df = pd.DataFrame([(encodings[m],better_interval_dir_names[direction],alt_mode_confidence_data[m][direction]) for m in encodings for direction in alt_mode_confidence_data[m]],columns=["Encoding","Performance","Count"])
alt_confidence_df

In [ ]:
plt.figure(figsize=(12,4))
sns.set_theme(style="whitegrid")
ax = sns.barplot(alt_confidence_df,x="Encoding",y="Count", hue="Performance", hue_order=["worse","similar","better"],palette = {
    'similar': 'tab:blue',
    'better': 'tab:green',
    'worse': 'tab:orange',
}
)
ax.set_ylabel("Number of values\n(across all metrics)")
ax.set_title("Performance of alternative approach compared to original")
sns.move_legend(ax,loc=(1.01, 0.6 ))
plt.savefig("{figure_path}/performance-comparison-alt-approach.pdf",bbox_inches='tight')


In [ ]:
import pandas as pd
values = []
for log_name in logs:
    for m in encodings:
        for d in all_results[log_name][m]:
            values.append({"log_name": log_name,"Encoding": encodings[m],**d['test_res'],**d['parameters']})
df = pd.DataFrame(values)
df

In [ ]:
hps = [d['parameters']  for log_name in logs for m in encodings for d in all_results[log_name][m]]
hps_per_mode = {m: d['parameters'] for m in encodings for log_name in logs  for d in all_results[log_name][m]}

sns.set_theme(style="whitegrid")
sns.set_palette("pastel")
sns.histplot(df,alpha=1.0, discrete=True, shrink=.8, hue="Encoding", x="layers", multiple='stack', legend=None)
ax = plt.gca()
# sns.move_legend(ax,loc=(1.01, 0.2))
ax.set_title("Number of layers")
ax.set_xlabel("Number of layers")
# ax.set_xticks([1,2,3,4])
# ax.set_yticks([i for i in range(0,22,2)])
# ax.set_xlim([0,5])
plt.savefig(f"{figure_path}/layer-sizes.pdf",bbox_inches='tight')
plt.show()

sns.histplot(df,alpha=1.0, discrete=True, shrink=.8, hue="Encoding", x="activation", multiple='stack')
ax = plt.gca()
sns.move_legend(ax,loc=(1.01, 0.2))
ax.set_title("Activation functions")
ax.set_xlabel("Activation functions")
plt.savefig(f"{figure_path}/activation-funs.pdf",bbox_inches='tight')
plt.show()

sns.histplot(df,alpha=1.0, hue="Encoding", x="learning_rate", multiple='stack',log_scale=True)
plt.gcf().set_figwidth(15)
ax = plt.gca()
ax.set_title("Learning rate")
ax.set_xlabel("Learning rate")
plt.savefig(f"{figure_path}/learning-rate.pdf",bbox_inches='tight')
plt.show()

sns.histplot(df,alpha=1.0, binwidth=30, hue="Encoding", x="units_0", multiple='stack')
ax = plt.gca()
sns.move_legend(ax,loc=(1.01, 0.2))
ax.set_title("Units in first layer")
ax.set_xlabel("Units in first layer")
plt.savefig(f"{figure_path}/units-in-layer-0_per_encoding.pdf",bbox_inches='tight')
plt.show()


sns.histplot([h['units_0'] for h in hps], kde=True, alpha=1.0, binwidth=15)
ax = plt.gca()
ax.set_title("Units in first layer")
ax.set_xlabel("Units in first layer")
plt.savefig(f"{figure_path}/units-in-layer-0.pdf",bbox_inches='tight')
plt.show()

sns.histplot([h['dropoutrate_0'] for h in hps], kde=True, alpha=1.0, binwidth=0.1)
ax = plt.gca()
ax.set_title("Dropout rate in first layer")
ax.set_xlabel("Dropout rate in first layer")
plt.savefig(f"{figure_path}/dropout-in-layer-0.pdf",bbox_inches='tight')
plt.show()


In [ ]:
hps_with_two_layers = [p for p in hps if p['layers'] >= 2]
sns.histplot([h['units_1'] for h in hps_with_two_layers], kde=True, alpha=1.0, binwidth=15)
ax = plt.gca()
ax.set_title("Units in second layer")
plt.savefig(f"{figure_path}/units-in-layer-1_{log_name}.pdf")
plt.show()

sns.histplot([h['dropoutrate_1'] for h in hps_with_two_layers], kde=True, alpha=1.0, binwidth=0.1)
ax = plt.gca()
ax.set_title("Dropout rate in second layer")
plt.savefig(f"{figure_path}/dropout-in-layer-1_{log_name}.pdf")
plt.show()